<a href="https://colab.research.google.com/github/kamalova/Capstone/blob/main/Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

# Data visualization
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
# Seaborn's beautiful styling
import seaborn as sns
# to get rid of the warnings
import warnings
warnings.filterwarnings("ignore")
sns.set_style('whitegrid')
from collections import Counter
from scipy import stats

from scipy.linalg import sqrtm
from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
%rm -rf sample_data/

In [11]:
df_reviews = pd.read_csv('/content/reviews_polarity.csv')
df_reviews

,listing_id,id,reviewer_id,reviewer_name,comments,month,weekday,language,text_length,polarity_score,sentiment_type
0,2595,19760,38960,Anita,i ve stayed with my friend at the midtown castle for six days and it was a lovely place to be a big spacious room with a pointy roof which really makes you feel like staying in a castle the location is perfect it is just a few steps from macy s time square and theatre district everything worked just perfect with the keys etc thank you so much jennifer we had a great time in new york attention it s on the floor without a lift but definetely worth it,12,Thursday,en,468,0.9274,Positive
1,2595,34320,71130,Kai-Uwe,we ve been staying here for about nights enjoying to be in the center of the city that never sleeps short ways to everywhere in manhattan by subway or by walk midtown castle is a beauftiful and tastful place jennifer and tori relaxed and friendly hosts thats why we the three berliners recommand that place good to have wifi and a little kitchen too,4,Friday,en,366,0.9136,Positive
2,2595,46312,117113,Alicia,we had a wonderful stay at jennifer s charming apartment they were very organized and helpful i would definitely recommend staying at the midtown castle,5,Tuesday,en,155,0.9409,Positive
3,2595,1238204,1783688,Sergey,hi to everyone would say our greatest compliments to jennifer the host of midtown castle we spent in this lovely apartment in the heart of manhattan one month april and will remember this time as ours best the apartment is pretty spacious and great located the th ave right around the corner there is everything you can need during your short or long stay jennifer is very friendly vigorous and very responsible host thanks her and highly recomend this apartment for everyone who are looking for a quiet place right in the center of the boiling midtown,5,Monday,en,570,0.9863,Positive
4,2595,1293632,1870771,Loïc,jennifer was very friendly and helpful and her place is exactly as advertised the location is very convenient and it was a pleasure to stay at the midtown castle i definitely recommend it thanks,5,Thursday,en,204,0.9542,Positive
...,...,...,...,...,...,...,...,...,...,...,...
70806,72265,161050979,109542482,John,vanessa was very pleasant and communication was very good,6,Friday,en,58,0.7774,Positive
70807,72265,163401732,1282541,Sofia,great location close to g train,6,Saturday,en,34,0.6249,Positive
70808,72265,252657179,8936723,Yo,highly recommend cannot beat this value great location minute walk to subway and sec to bus which connects you easily and quickly to various parts of manhattan and brooklyn organic as well as regular grocery stores and lots of awesome restaurants and stores near by very safe neighborhoods nice room not big but it s plenty enough and everything works well it s nice warm in the winter even though the bedroom is separated by a curtain to the kitchen because the host is mainly in the other section of the apartment you have a lot of privacy vanessa is a very friendly interesting and helpful host,4,Wednesday,en,626,0.9870,Positive
70809,72265,277084426,17160406,Ioannis,vanessa is a great and very polite host and gives you as much privacy as you want the room can be seen in the photos and has everything you need the location is amazing as well with plenty of bars restaurants and stores around and literally half a block away from g train,6,Friday,en,275,0.8126,Positive


In [12]:
df_reviews.columns

Index(['listing_id', 'id', 'reviewer_id', 'reviewer_name', 'comments', 'month',
       'weekday', 'language', 'text_length', 'polarity_score',
       'sentiment_type'],
      dtype='object')

In [51]:
df_reviews.drop(columns=['id','comments','month','weekday','language','text_length'],inplace=True)

In [52]:
df_reviews

,listing_id,reviewer_id,reviewer_name,polarity_score,sentiment_type
0,2595,38960,Anita,0.9274,Positive
1,2595,71130,Kai-Uwe,0.9136,Positive
2,2595,117113,Alicia,0.9409,Positive
3,2595,1783688,Sergey,0.9863,Positive
4,2595,1870771,Loïc,0.9542,Positive
...,...,...,...,...,...
70806,72265,109542482,John,0.7774,Positive
70807,72265,1282541,Sofia,0.6249,Positive
70808,72265,8936723,Yo,0.9870,Positive
70809,72265,17160406,Ioannis,0.8126,Positive


In [14]:
df_reviews.polarity_score.describe()

count    70811.000000
mean         0.877807
std          0.205164
min         -0.995000
25%          0.872000
50%          0.945100
75%          0.974700
max          0.999400
Name: polarity_score, dtype: float64

In [16]:
! pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 771 kB 22.5 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-linux_x86_64.whl size=2626491 sha256=ac6fb31c5a7a908a73dfe82b5be451930d47a0abf8bdd5c681ffe5213054198f
  Stored in directory: /root/.cache/pip/wheels/af/db/86/2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built scikit-surprise


#### Building Recommender System 

The recommender systems will be built using surprise package (Matrix Factorization - based models).



**SVD and NMF** models comparison <p>
Singular Value Decomposition (SVD) and Non-negative Matrix Factorization (NMF) are matrix factorization techniques used for dimensionality reduction. Surprise package provides implementation of those algorithms.

It's clear that for the given dataset much better results can be obtained with SVD approach - both in terms of accuracy and training / testing time.

In [36]:
from surprise import SVD, NMF, KNNBasic, Dataset, Reader, accuracy
from surprise.model_selection import cross_validate, train_test_split, GridSearchCV

In [18]:
reader = Reader(rating_scale=(-1,1))

In [22]:
df = Dataset.load_from_df(df_reviews[['listing_id', 'reviewer_id', 'polarity_score']], reader)


In [33]:
model_svd = SVD()
cv_results_svd = cross_validate(model_svd, df, cv=5)
pd.DataFrame(cv_results_svd).mean()


test_rmse    0.154272
test_mae     0.085050
fit_time     1.033165
test_time    0.111200
dtype: float64

In [35]:
model_nmf = NMF()
cv_results_nmf = cross_validate(model_nmf, df, cv=5)
pd.DataFrame(cv_results_nmf).mean()


test_rmse    0.103816
test_mae     0.069779
fit_time     1.913741
test_time    0.145357
dtype: float64

**Optimisation of SVD Algorithm** <p>
Grid Search Cross Validation computes accuracy metrics for an algorithm on various combinations of parameters, over a cross-validation procedure. It's useful for finding the best configuration of parameters.<p>

It is used to find the best setting of parameters:

- n_factors - the number of factors
- n_epochs - the number of iteration of the SGD procedure
- lr_all - the learning rate for all parameters
- reg_all - the regularization term for all parameters<p>
As a result, regarding the majority of parameters, the default setting is the most optimal one. The improvement obtained with Grid Search is very small.

In [37]:
param_grid = {'n_factors': [80,100,120],
              'n_epochs': [5, 10, 20],
              'lr_all': [0.002, 0.005],
              'reg_all': [0.2, 0.4, 0.6]}

GS = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
GS.fit(df)

print(GS.best_score['rmse'])
print(GS.best_params['rmse'])


0.1715642196945499
{'n_factors': 120, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.2}


**Analysis of Collaborative Filtering Model Results**<p>
In this part, let's examine in detail the results obtained by the SVD model that provided the best RMSE score.

In [39]:
train_set, test_set = train_test_split(df, test_size=0.2)

model_svd = SVD(n_factors=120, n_epochs=20, lr_all=0.005, reg_all=0.2)
model_svd.fit(train_set)
predictions = model_svd.test(test_set)


In [68]:
print('Accuracy on test data set,', end='   ')
accuracy.rmse(predictions,verbose=True)


Accuracy on test data set,   RMSE: 0.1630


0.16296925179340024

In [47]:
df_pred = pd.DataFrame(predictions, columns=['listing_id', 'reviewer_id', 'polarity_score' ,'pred_pol', 'details'])

df_pred['impossible'] = df_pred['details'].apply(lambda x: x['was_impossible'])
df_pred['pred_pol_round'] = df_pred['pred_pol'].round()
df_pred['abs_err'] = abs(df_pred['pred_pol'] - df_pred['polarity_score'])
df_pred.drop(['details'], axis=1, inplace=True)

df_pred.sample(5)


,listing_id,reviewer_id,polarity_score,pred_pol,impossible,pred_pol_round,abs_err
4021,56859,9263705,0.9606,0.890702,False,1.0,0.069898
11967,27644,20438048,0.9378,0.881811,False,1.0,0.055989
1847,60457,290583,0.9665,0.939446,False,1.0,0.027054
4987,54860,1584919,0.9890,0.911533,False,1.0,0.077467
2482,68974,44074351,0.7650,0.875984,False,1.0,0.110984


In [62]:
df_reviews[df_reviews['pred_pol'].notna()].sort_values('polarity_score', ascending=False).head(10)

,listing_id,reviewer_id,reviewer_name,polarity_score,sentiment_type,pred_pol
17589,64314,968402,Sjoerd,0.9994,Positive,0.955771
59270,64314,968402,Sjoerd,0.9994,Positive,0.955771
36924,64314,968402,Sjoerd,0.9994,Positive,0.955771
80692,64314,968402,Sjoerd,0.9994,Positive,0.955771
80691,64314,968402,Sjoerd,0.9994,Positive,0.955771
17588,64314,968402,Sjoerd,0.9994,Positive,0.955771
59269,64314,968402,Sjoerd,0.9994,Positive,0.955771
36925,64314,968402,Sjoerd,0.9994,Positive,0.955771
22915,80493,23967535,Ilona,0.9992,Positive,0.862491
72468,32037,17392875,Karen,0.9992,Positive,0.923954


In [79]:
from collections import defaultdict

def get_top_recommendations(predictions, topN = 3):
     
    top_recs = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_recs[uid].append((iid, est))
     
    for uid, user_ratings in top_recs.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_recs[uid] = user_ratings[:topN]    
    return top_recs
print('getting top 10 recommendations')

getting top 10 recommendations


In [86]:
top_10 = get_top_recommendations(predictions,3)

In [90]:

df_ex = pd.DataFrame(columns=['UserId', 'Recommended Listing - Rating'])
i=0;
for uid, user_ratings in top_10.items():
    #print(uid, top3_recommendations[uid])
    row = [uid, top_10[uid]]
    df_ex.loc[i] = row
    i=i+1
#dfo.to_csv('suissionF.csv', index = False)

print("wrote Recommendations for each user in csv file")   
 

wrote Recommendations for each user in csv file


In [91]:
df_ex.head()

,UserId,Recommended Listing - Rating
0,68974,"[(56114910, 0.9249555075628081), (15289448, 0.9242583060754467), (97286063, 0.9237735014807139)]"
1,23135,"[(10768644, 0.9158114164603399), (5789061, 0.913662504298596), (5789061, 0.913662504298596)]"
2,12937,"[(1004803, 0.9429439695182918), (122451751, 0.9290618802110597), (22826920, 0.9288914648424864)]"
3,29013,"[(4260506, 0.904797363308971), (3389370, 0.8901609289954896), (13677999, 0.8859731543123567)]"
4,21456,"[(94354, 0.9244170742666037), (265051888, 0.9207863500091971), (1430910, 0.9188960837435869)]"
